# **Previsão de preço de carros.**
<!--
<img src="https://miro.medium.com/v2/resize:fit:988/0*tA5OjppLK627FfFo" alt="image" width="250" height="auto">
-->
Nesse projeto eu vou construir um modelo de **Machine Learning** que faça a previsão  do preço de venda de um veículo.

Irei utilizar a Metodologia **CRISP-DM** para me basear nos passos a serem seguinos no projeto.

1. **Compreensão do Negócio (Business Understanding/Problem)** 
1. **Compreensão dos Dados (Data Understanding)** 
1. **Preparação dos Dados (Data Preparation)** 
1. **Exploração dos dados (EDA)**
1. **Engenharia de features, Data Transformation e Pré-processamento**
1. **Modelagem (Modeling)** 
1. **Avaliação (Evaluation)**
1. **Conclusão e interpretação dos resultados.**
1. **Implantação (Deployment)**

### **Importando Bibliotecas**

In [47]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from platform import python_version
print('Versão da Linguagem Python:', python_version())

Versão da Linguagem Python: 3.12.5


In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
### Visualization Settings
%matplotlib inline

# Filter warnings.
import warnings
warnings.filterwarnings('ignore')

mpl.style.use('ggplot') 
plt.style.use('meu_estilo.mplstyle')

# Display options.
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.width', 1000)

### **Lendo Dataset**

In [50]:
path = '../data_lake/silver/dados_veiculos_cleaned.csv'
df = pd.read_csv(path)

In [51]:
df.sample(3)

,titulo,preco_brl,descricao,opcionais,cep,imagem,perfil_carro,data_publicacao,tempo_publicacao,cod_publicacao,quilometragem,possui_kit_gnv,tipo_de_veiculo,cambio,cor,tipo_de_direção,potencia_do_motor,portas,modelo,ano,marca,combustivel,qtd_opcionais
8078,Hyundai Hb20 2022 1.0 12v flex evolution manual,66990,HYUNDAI HB20 1.0 12V FLEX EVOLUTION MANUAL - 2...,"Alarme, Ar condicionado, Trava elétrica",8735200,https://img.olx.com.br/images/67/6784222027091...,https://sp.olx.com.br/sao-paulo-e-regiao/autos...,2024-09-13,10:48,1335307292,51684,Sim,Hatch,Manual,Prata,Elétrica,1.0,4,HYUNDAI HB20 EVOLUTION 1.0 FLEX 12V MEC,2022,HYUNDAI,Flex,3
21250,"Gol g3 1.0 , 2005 flex",12000,"Carro precisa de uns serviços como suspensão, ...","Ar condicionado, Alarme, Som",51160260,https://img.olx.com.br/images/59/5984824589864...,https://pe.olx.com.br/grande-recife/autos-e-pe...,2024-10-02,13:23,1342564157,190000,Não,Hatch,Manual,Cinza,Mecânica,1.0,4,VOLKSWAGEN GERAÇÃO III CITY 1.0 MI 8V TOTAL FL...,2005,VOLKSWAGEN,Flex,3
3323,Mitsubishi Eclipse cross 2023 1.5 mivec turbo ...,135090,MITSUBISHI ECLIPSE CROSS 1.5 MIVEC TURBO GASOL...,Ar condicionado,44050794,https://img.olx.com.br/images/26/2654327934004...,https://ba.olx.com.br/regiao-de-feira-de-santa...,2024-09-09,18:40,1299354570,43786,Não,SUV,Automático,Branco,Elétrica,1.5,4,MITSUBISHI HPE 1.5 16V 165CV AUT,2023,MITSUBISHI,Gasolina,1


# **5. Engenharia de features, Data Transformation e Pré-processamento**

Primeiramente irei resolver alguns problemas observados anteriormente na etapa `4. EDA`, como tratar outliers em algumas variáveis e também tratar veículos com **preços** de **0 R$** caso seja necessário.

## **Removendo Outliers**

A técnica para idêntificação e remoção de outliers utilizada será o ***método de intervalo interquartil***. Eu decidi por **remover os Outliers** ao invés de **substituir** por outro valor porque mesmo após remoção ainda haverá uma boa quantidade de dados para serem utilizados.

In [56]:
def excluir_outliers(data, coluna):
    q1 = data[coluna].quantile(0.25)
    q3 = data[coluna].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    data = data.loc[(data[coluna] >= lower) & (data[coluna] <= upper), :]
    return data

Removendo Outliers das variáveis `quilometragem` e `preco_brl`.

In [54]:
qtd_linhas = df.shape[0]
df2 = excluir_outliers(df, coluna='quilometragem')
df_s_outliers = excluir_outliers(df2, coluna='preco_brl')
linhas_removidas = qtd_linhas - df_s_outliers.shape[0]
print(f'Foram removidas um total de {linhas_removidas} linhas.')

Foram removidas um total de 3263 linhas.
